In [59]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer


from category_encoders import TargetEncoder


In [60]:
df_train = pd.read_csv("train.csv")
df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [61]:
corr_matrix = df_train.corr(numeric_only=True)
saleprice_corr = corr_matrix["SalePrice"].sort_values(ascending=False)


In [62]:
saleprice_corr.head(35)

SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
YearRemodAdd    0.507101
GarageYrBlt     0.486362
MasVnrArea      0.477493
Fireplaces      0.466929
BsmtFinSF1      0.386420
LotFrontage     0.351799
WoodDeckSF      0.324413
2ndFlrSF        0.319334
OpenPorchSF     0.315856
HalfBath        0.284108
LotArea         0.263843
BsmtFullBath    0.227122
BsmtUnfSF       0.214479
BedroomAbvGr    0.168213
ScreenPorch     0.111447
PoolArea        0.092404
MoSold          0.046432
3SsnPorch       0.044584
BsmtFinSF2     -0.011378
BsmtHalfBath   -0.016844
MiscVal        -0.021190
Id             -0.021917
LowQualFinSF   -0.025606
YrSold         -0.028923
OverallCond    -0.077856
Name: SalePrice, dtype: float64

In [63]:
df_train.info()

<class 'pandas.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   str    
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   str    
 6   Alley          91 non-null     str    
 7   LotShape       1460 non-null   str    
 8   LandContour    1460 non-null   str    
 9   Utilities      1460 non-null   str    
 10  LotConfig      1460 non-null   str    
 11  LandSlope      1460 non-null   str    
 12  Neighborhood   1460 non-null   str    
 13  Condition1     1460 non-null   str    
 14  Condition2     1460 non-null   str    
 15  BldgType       1460 non-null   str    
 16  HouseStyle     1460 non-null   str    
 17  OverallQual    1460 non-null   int64  
 18  OverallCond    1460

In [64]:
len(df_train.Alley.unique())

3

In [65]:
df_train.Exterior2nd.unique()

<StringArray>
['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng', 'CmentBd',
 'BrkFace',  'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc', 'AsphShn',   'Stone',
   'Other',  'CBlock']
Length: 16, dtype: str

In [73]:
print("Skewness:", df_train["SalePrice"].skew())


Skewness: 1.8828757597682129


In [74]:
df_train["SalePrice"] = np.log1p(df_train["SalePrice"])


In [75]:
df_train["Neighborhood"].value_counts()


Neighborhood
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     28
StoneBr     25
SWISU       25
MeadowV     17
Blmngtn     17
BrDale      16
Veenker     11
NPkVill      9
Blueste      2
Name: count, dtype: int64

In [77]:
def group_rare_categories(df, col, threshold=0.01):
    freq = df[col].value_counts(normalize=True)
    rare_labels = freq[freq < threshold].index
    df[col] = df[col].replace(rare_labels, "Other")
    return df
for col, dtype in df_train.dtypes.items():
    if dtype == "str":
        df_train = group_rare_categories(df_train, col, threshold=0.01)


In [78]:
y = df_train["SalePrice"]
X = df_train.drop(columns=["SalePrice", "Id"])

In [80]:

numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

one_hot = []
target_encoding = []

for col, dtype in X.dtypes.items():
    if dtype=="str" and len(X[col].unique()) <= 15:
        one_hot.append(col)
    elif dtype=="str" and len(X[col].unique()) > 15:
        target_encoding.append(col)
print("This onehot: ",one_hot)
print("And This is target encoding: ", target_encoding)


This onehot:  ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
And This is target encoding:  ['Neighborhood']


In [81]:

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

onehot_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))
])

target_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("target", TargetEncoder())
])


In [82]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numeric_cols),
        ("onehot", onehot_pipeline, one_hot),
        ("target", target_pipeline, target_encoding)
    ]
)


In [83]:
model = XGBRegressor(
    objective="reg:squarederror",
    random_state=42
)


In [84]:
pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("model", model)
])


In [97]:
param_grid = {
    "model__n_estimators": [500, 1000],
    "model__max_depth": [3, 4, 5],
    "model__learning_rate": [0.01, 0.05, 0.1],
    "model__subsample": [0.8, 1.0]
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",  # cleaner
    n_jobs=-1
)

grid.fit(X, y)
best_model = grid.best_estimator_

print("Best parameters:", grid.best_params_)
print("Best CV RMSE:", -grid.best_score_)


/home/aryan/miniconda3/envs/house-price-estimate/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [21, 26] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/home/aryan/miniconda3/envs/house-price-estimate/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [39] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/home/aryan/miniconda3/envs/house-price-estimate/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [37, 39] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/home/aryan/miniconda3/envs/house-price-estimate/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in

Best parameters: {'model__learning_rate': 0.05, 'model__max_depth': 3, 'model__n_estimators': 500, 'model__subsample': 0.8}
Best CV RMSE: 0.12073609871613515


In [98]:
df_test = pd.read_csv("test.csv")
df_test.columns


Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [99]:
test_ids = df_test["Id"]
df_test = df_test.drop(columns=["Id"])


In [100]:
test_predictions = best_model.predict(df_test)
test_predictions = np.expm1(test_predictions)


/home/aryan/miniconda3/envs/house-price-estimate/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0, 1, 3, 6, 7, 8, 9, 11, 12, 13, 14, 15, 17, 18, 19, 21, 24, 25, 26, 28, 30, 32, 34, 35, 40, 41] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


In [101]:
test_predictions

array([126173.17, 153190.72, 184066.94, ..., 172583.69, 112482.18,
       209550.66], shape=(1459,), dtype=float32)

In [102]:
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": test_predictions
})

submission.to_csv("submission.csv", index=False)
